# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Word2Vec
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
from numba import jit
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler, LabelEncoder
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

In [5]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)

print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 11)
Daum : (9372, 11)


In [6]:
extNaverData = naverData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extNaverData['site'] = pd.Series(['Naver']*extNaverData.shape[0])
extDaumData = daumData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extDaumData['site'] = pd.Series(['daum']*extDaumData.shape[0])

In [7]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    #newsPath = '/Volumes/data/pre_data/news_sentiment/'
    newsPath = './data/pre_data/news_sentiment/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'

## TaggedDocument

In [8]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Word2Vec Model

### Twitter

In [9]:
taggerName = 'ct'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [10]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_ct.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:00<00:00, 1350491.88it/s]


(442359, 159113)
vocab size : 159113


#### News to tagged Document

In [11]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, ct, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [12]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### model1

In [13]:
modelName = bm.Return_ModelName('word2vec', model1,'ct')

In [14]:
classifierList = glob(classifierPath+'*'+modelName)

In [15]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

XGBoost
NeuralNetwork_2
NeuralNetwork_1
LogisticRegression
SVC
RandomForestClassifier


In [16]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

  7%|▋         | 11609/162640 [00:00<00:01, 116060.23it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 126897.24it/s]
1it [00:00,  6.71it/s]

running time : 0:00:01.285829
Vectorizing Data


9372it [01:26, 107.83it/s]


scaling Data
total running time : 0:01:28.657895


In [17]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 323618.90it/s]
9372it [00:00, 828063.81it/s]
9372it [00:00, 665104.68it/s]


CPU times: user 2.75 s, sys: 451 ms, total: 3.2 s
Wall time: 3.09 s


In [18]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

  9%|▉         | 14877/162640 [00:00<00:00, 148728.30it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 119106.91it/s]
7it [00:00, 66.30it/s]

running time : 0:00:01.370038
Vectorizing Data


15120it [02:34, 98.16it/s]


scaling Data
total running time : 0:02:36.194049


In [19]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 782289.67it/s]
15120it [00:00, 828352.99it/s]
15120it [00:00, 731665.93it/s]


CPU times: user 4.28 s, sys: 642 ms, total: 4.92 s
Wall time: 4.52 s


In [20]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [21]:
modelName = bm.Return_ModelName('word2vec', model2,'ct')

In [22]:
classifierList = glob(classifierPath+'*'+modelName)

In [23]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [24]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

  5%|▍         | 7960/162640 [00:00<00:01, 79580.91it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 123046.11it/s]
2it [00:00, 15.44it/s]

running time : 0:00:01.325363
Vectorizing Data


9372it [01:14, 125.45it/s]


scaling Data
total running time : 0:01:16.454055


In [25]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 190271.82it/s]
9372it [00:00, 804177.84it/s]
9372it [00:00, 809527.10it/s]


CPU times: user 26 s, sys: 990 ms, total: 26.9 s
Wall time: 25.9 s


In [26]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

  8%|▊         | 12888/162640 [00:00<00:01, 128837.43it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 117292.43it/s]
19it [00:00, 93.81it/s]

running time : 0:00:01.390565
Vectorizing Data


15120it [02:19, 108.07it/s]


scaling Data
total running time : 0:02:22.047837


In [27]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 745890.84it/s]
15120it [00:00, 869416.89it/s]
15120it [00:00, 731429.65it/s]


CPU times: user 41.5 s, sys: 1.85 s, total: 43.4 s
Wall time: 41.2 s


In [28]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [29]:
modelName = bm.Return_ModelName('word2vec', model3,'ct')

In [30]:
classifierList = glob(classifierPath+'*'+modelName)

In [31]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [32]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

  5%|▍         | 7850/162640 [00:00<00:01, 78481.36it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 119849.09it/s]
11it [00:00, 97.77it/s]

running time : 0:00:01.360659
Vectorizing Data


9372it [01:14, 125.33it/s]


scaling Data
total running time : 0:01:16.461639


In [33]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 807996.24it/s]
9372it [00:00, 795503.64it/s]
9372it [00:00, 875167.36it/s]


CPU times: user 29.1 s, sys: 561 ms, total: 29.7 s
Wall time: 28.5 s


In [34]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

  8%|▊         | 12263/162640 [00:00<00:01, 122590.66it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 125470.70it/s]
9it [00:00, 85.33it/s]

running time : 0:00:01.300000
Vectorizing Data


15120it [02:19, 108.21it/s]


scaling Data
total running time : 0:02:21.844667


In [35]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 781431.77it/s]
15120it [00:00, 735765.97it/s]
15120it [00:00, 515089.96it/s]


CPU times: user 46.3 s, sys: 1.04 s, total: 47.4 s
Wall time: 45.9 s


In [36]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

### Mecab

In [37]:
taggerName = 'mecab'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [38]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:00<00:00, 590832.81it/s] 


(442359, 161590)
vocab size : 161590


#### News to tagged Document

In [39]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, mecab, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [40]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### model1

In [41]:
modelName = bm.Return_ModelName('word2vec', model1,'mecab')

In [42]:
classifierList = glob(classifierPath+'*'+modelName)

In [43]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [44]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

  6%|▌         | 9203/162640 [00:00<00:01, 91957.30it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 101797.54it/s]
1it [00:00,  7.66it/s]

running time : 0:00:01.607909
Vectorizing Data


9372it [01:41, 92.31it/s]


scaling Data
total running time : 0:01:43.553051


In [45]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 259820.46it/s]
9372it [00:00, 735407.79it/s]
9372it [00:00, 803783.19it/s]


CPU times: user 23 s, sys: 1.77 s, total: 24.8 s
Wall time: 23.4 s


In [46]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

  5%|▌         | 8902/162640 [00:00<00:01, 88989.96it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 101402.65it/s]
6it [00:00, 51.65it/s]

running time : 0:00:01.607507
Vectorizing Data


15120it [02:38, 95.46it/s]


scaling Data
total running time : 0:02:40.841778


In [47]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 617169.57it/s]
15120it [00:00, 726336.32it/s]
15120it [00:00, 803612.40it/s]


CPU times: user 35.8 s, sys: 1.95 s, total: 37.8 s
Wall time: 35.6 s


In [48]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [49]:
modelName = bm.Return_ModelName('word2vec', model2,'mecab')

In [50]:
classifierList = glob(classifierPath+'*'+modelName)

In [51]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [52]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

  6%|▌         | 9658/162640 [00:00<00:01, 96540.27it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 124614.60it/s]
3it [00:00, 27.80it/s]

running time : 0:00:01.309411
Vectorizing Data


9372it [01:18, 119.11it/s]


scaling Data
total running time : 0:01:20.326223


In [53]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 776919.46it/s]
9372it [00:00, 888921.94it/s]
9372it [00:00, 821384.90it/s]


CPU times: user 24.6 s, sys: 972 ms, total: 25.6 s
Wall time: 24.4 s


In [54]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

  6%|▌         | 10024/162640 [00:00<00:01, 100209.99it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 135940.95it/s]
6it [00:00, 55.51it/s]

running time : 0:00:01.200323
Vectorizing Data


15120it [02:28, 101.77it/s]


scaling Data
total running time : 0:02:30.622805


In [55]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 638315.05it/s]
15120it [00:00, 895947.85it/s]
15120it [00:00, 738740.03it/s]


CPU times: user 39 s, sys: 706 ms, total: 39.7 s
Wall time: 38.2 s


In [56]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [57]:
modelName = bm.Return_ModelName('word2vec', model3,'mecab')

In [58]:
classifierList = glob(classifierPath+'*'+modelName)

In [59]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [60]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

  8%|▊         | 13263/162640 [00:00<00:01, 132581.13it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 117969.71it/s]
11it [00:00, 103.56it/s]

running time : 0:00:01.383561
Vectorizing Data


9372it [01:21, 115.67it/s]


scaling Data
total running time : 0:01:22.734226


In [61]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 797213.78it/s]
9372it [00:00, 837377.61it/s]
9372it [00:00, 820168.11it/s]


CPU times: user 29.1 s, sys: 897 ms, total: 30 s
Wall time: 28.8 s


In [62]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

  8%|▊         | 12755/162640 [00:00<00:01, 127505.13it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 115133.40it/s]
5it [00:00, 47.16it/s]

running time : 0:00:01.416188
Vectorizing Data


15120it [02:28, 101.82it/s]


scaling Data
total running time : 0:02:30.576347


In [63]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 802341.53it/s]
15120it [00:00, 867929.55it/s]
15120it [00:00, 814867.48it/s]


CPU times: user 46.3 s, sys: 625 ms, total: 46.9 s
Wall time: 45.5 s


In [64]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict